In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import glob
import keras
from functools import reduce

Using TensorFlow backend.
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tenso

In [2]:
path = r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Windowed Data' # use your path
all_files = glob.glob(path + "/*.csv")

In [3]:
li=[]
frame=[]

In [4]:
for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    df1['hr']= df1['hr'].astype(float)
    hrmean=df1.groupby('windowno', as_index=False)['hr'].mean()
    hrmean=hrmean.rename(columns={"hr": "hrmean"})
    hrsd=df1.groupby('windowno').agg(np.std,ddof=0)['hr']
    hrsd=pd.DataFrame(hrsd)
    hrsd=hrsd.rename(columns={"hr": "hrsd"})
    stress=df1.groupby('windowno', as_index=False)['ptsd_moment'].max()
    hrmin=df1.groupby('windowno', as_index=False)['hr'].min()
    hrmin=hrmin.rename(columns={"hr": "hrmin"})
    hrmax=df1.groupby('windowno', as_index=False)['hr'].max()
    hrmax=hrmax.rename(columns={"hr": "hrmax"})
    participant = df1['participant'].iloc[0]
    data=reduce(lambda x,y: pd.merge(x,y, on='windowno', how='outer'),
       [hrmean,hrmin,hrmax,hrsd,stress])
    data['participant']= participant
    li.append(data)
frame = pd.concat(li, axis=0, ignore_index=True)

In [5]:
frame['hrrange']= frame['hrmax']-frame['hrmin']

In [125]:
frame.to_csv (r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/HR Features/HR Features_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [6]:
frame

windowno      hrmean  hrmin  hrmax       hrsd  ptsd_moment  \
0          2914   66.708347   62.0   73.0   3.040741          1.0   
1          2915   71.868832   64.0   80.0   4.653403          1.0   
2          2917   85.688151   82.0   91.0   2.840909          1.0   
3          2918   88.736465   85.0   92.0   2.299090          1.0   
4          2922   84.311374   77.0   91.0   4.667418          0.0   
...         ...         ...    ...    ...        ...          ...   
13339     10624   62.202333   59.0   66.0   2.620253          0.0   
13340     10625   63.031590   58.0   67.0   2.574690          0.0   
13341     11159  110.224178  105.0  113.0   2.229094          0.0   
13342     11413   95.140097   91.0   98.0   2.525024          0.0   
13343     12609  102.244495   62.0  112.0  10.384461          0.0   

       participant  hrrange  
0               13     11.0  
1               13     16.0  
2               13      9.0  
3               13      7.0  
4               13     14.0  
...            ...      ...  
13339            3      7.0  
13340            3      9.0  
13341            3      8.0  
13342            3      7.0  
13343            3     50.0  

[13344 rows x 8 columns]